In [169]:
import pandas as pd

df_cultura = pd.read_csv("../results_50/raw_results_north_south_china.csv", index_col = [0])
df_cultura['N_est'] = df_cultura['f0'] + df_cultura['f1'] + df_cultura['f2']# plus the individual
df_cultura['N_est_min']  =  df_cultura['min_f0'] + df_cultura['f1'] + df_cultura['f2']# plus the individual
df_cultura['N_est_max']  =  df_cultura['max_f0'] + df_cultura['f1'] + df_cultura['f2']# plus the individual

print(len(set(df_cultura.individual_wikidata_id)))

df_cultura = df_cultura.groupby(['region_name', 'decade']).agg({'N_est': 'sum', 'N_est_min':'sum', 'N_est_max':'sum', 'individual_wikidata_id':'count'}).reset_index()
df_cultura = df_cultura.rename(columns={"individual_wikidata_id": "score"})
df_cultura = df_cultura[df_cultura['decade'] <=1800]

df_cultura_world = df_cultura.drop('region_name', axis=1).groupby('decade').sum().reset_index()
df_cultura_world['region_name']='world'

df_cultura = pd.concat([df_cultura_world, df_cultura])


95955


In [170]:
df_individuals = pd.read_csv('/Users/charlesdedampierre/Desktop/cultura_project/immaterial_index/results/df_individuals_score.csv', index_col = [0])
df_individuals = df_individuals[df_individuals['decade']<=1800]

# df_individuals = df_individuals[df_individuals['region_name'].isin(['Southern China', 'Northern China'])]
df_individuals = df_individuals[df_individuals['region_name'].isin(['Chinese world'])]
df_individuals.sort_values('score', ascending=False)

,individual_wikidata_id,productive_year,decade,individual_name,score,region_name
391963,Q4604,-515.0,-520,Confucius,152,Chinese world
573060,Q9333,-544.0,-540,Laozi,109,Chinese world
523643,Q7071,736.0,740,Li Bai,109,Chinese world
329193,Q33772,747.0,750,Du Fu,98,Chinese world
238684,Q233340,1587.0,1590,Matteo Ricci,94,Chinese world
...,...,...,...,...,...,...
553791,Q8249488,1647.0,1650,Yang Wan,1,Chinese world
553790,Q8249488,1644.0,1640,Yang Wan,1,Chinese world
205554,Q2087245,1405.0,1400,Hong Bao,1,Chinese world
461082,Q56760586,-200.0,-200,Mao Chang,1,Chinese world


# Index for Chinese World and Low Countries

In [171]:
df_cultura['survival_rate'] = df_cultura['score']/ df_cultura['N_est']
df_cultura['survival_rate_min'] = df_cultura['score']/ df_cultura['N_est_min']
df_cultura['survival_rate_max'] = df_cultura['score']/ df_cultura['N_est_max']

df_cultura

,decade,N_est,N_est_min,N_est_max,score,region_name,survival_rate,survival_rate_min,survival_rate_max
0,-800,9.226102,4.694602,98.180245,3,world,0.325164,0.639032,0.030556
1,-700,32.396682,17.957384,98.733649,5,world,0.154337,0.278437,0.050641
2,-650,21.058310,12.194590,54.557692,4,world,0.189949,0.328014,0.073317
3,-600,91.376628,57.523203,193.212908,13,world,0.142268,0.225996,0.067283
4,-550,174.685744,124.355749,282.260187,25,world,0.143114,0.201036,0.088571
...,...,...,...,...,...,...,...,...,...
547,1600,8055.617017,7188.343028,9143.776682,445,United Kingdom,0.055241,0.061906,0.048667
548,1650,7379.535596,6654.309550,8289.574452,472,United Kingdom,0.063961,0.070931,0.056939
549,1700,9815.101746,8787.851420,11106.756134,546,United Kingdom,0.055629,0.062131,0.049159
550,1750,16299.503550,14605.284410,18443.207874,946,United Kingdom,0.058039,0.064771,0.051293


In [172]:
list(set(df_cultura.region_name))

['Central Europe',
 'Southern China',
 'East Slavic',
 'Low countries',
 'Chinese world',
 'France',
 'Nordic countries',
 'German world',
 'Spain',
 'Japan',
 'Italy',
 'Latin World',
 'Persian world',
 'United Kingdom',
 'Indian world',
 'Northern China',
 'Greek World',
 'world',
 'Portugal',
 'Korea',
 'Arabic world']

In [173]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from statsmodels.nonparametric.smoothers_lowess import lowess

# Filtering logic
# df = df_cultura[df_cultura['region_name'].isin(['Southern China', 'Northern China'])]
df = df_cultura[df_cultura['region_name'].isin(['Chinese world'])]
df = df[df['decade'] >= -800]
df['N_est'] = np.log(df['N_est'])  # Applying log transformation
df['N_est_min'] = np.log(df['N_est_min'])  # Min bound log
df['N_est_max'] = np.log(df['N_est_max'])  # Max bound log

# Create figure
fig = go.Figure()

# List of unique regions to plot
regions = df['region_name'].unique()

# Colors for each line and their respective confidence interval fill
# colors = {
#     "Northern China": ("blue", "rgba(173,216,230,0.5)"),  # Light blue
#     "Southern China": ("red", "rgba(240,128,128,0.5)")   # Light red
# }


colors = {
     "Chinese world": ("blue", "rgba(173,216,230,0.5)"),  # Light blue  # Light red
 }

# Perform LOESS smoothing and plotting
for region in regions:
    # Extract data for the region
    region_data = df[df['region_name'] == region]
    region_data.sort_values(by='decade', inplace=True)  # Sort by decade for consistent plotting

    # Applying LOESS smoothing for mean and confidence intervals
    smoothed = lowess(region_data['N_est'], region_data['decade'], frac=0.2)
    smoothed_min = lowess(region_data['N_est_min'], region_data['decade'], frac=0.2)
    smoothed_max = lowess(region_data['N_est_max'], region_data['decade'], frac=0.2)

    decades, n_est = smoothed[:, 0], smoothed[:, 1]
    n_est_min = smoothed_min[:, 1]
    n_est_max = smoothed_max[:, 1]

    # Main line
    fig.add_trace(go.Scatter(
        x=decades,
        y=n_est,
        mode='lines',
        name=f"{region}",
        line=dict(color=colors[region][0], width=3),
    ))

    # Confidence interval fill
    fig.add_trace(go.Scatter(
        x=np.concatenate([decades, decades[::-1]]),  # x and then x reversed for fill
        y=np.concatenate([n_est_min, n_est_max[::-1]]),  # upper bound and lower bound reversed
        fill='toself',
        fillcolor=colors[region][1],
        line=dict(color='rgba(255,255,255,0)'),  # Transparent line for fill area
        hoverinfo="skip",
        showlegend=False
    ))

# Add scatter points for individuals
for region in regions:
    region_individuals = df_individuals[df_individuals['region_name'] == region]
    top_individuals = region_individuals.nlargest(15, 'score')

    # Normalize scores to match top of left y-axis
    max_score = top_individuals['score'].max()
    max_y = max(n_est)  # Maximum y-value from the LOESS smoothing

    top_individuals['normalized_score'] = top_individuals['score'] / max_score * max_y

    fig.add_trace(go.Scatter(
        x=top_individuals['decade'],
        y=top_individuals['normalized_score'],
        mode='markers+text',
        name=None,  # Hide name from legend
        text=top_individuals['individual_name'],
        textposition="top center",
        textfont=dict(
            color=colors[region][0]  # Set text color to match the line color
        ),
        marker=dict(color=colors[region][0]),
        showlegend=False
    ))

# Update layout
fig.update_layout(
    title='',
    xaxis_title='',
    yaxis_title='Corrected CPI (Log Scale)',
    template='plotly_white',  # Use a clean and clear template
    width=1100,  # Width in pixels
    height=900   # Height in pixels
)

fig.show()


In [198]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from statsmodels.nonparametric.smoothers_lowess import lowess




# Filtering logic
df = df_cultura[df_cultura['region_name'].isin(['Chinese world'])]
df = df[df['decade'] >= -800]
df['N_est'] = np.log(df['N_est'])  # Applying log transformation
df['N_est_min'] = np.log(df['N_est_min'])  # Min bound log
df['N_est_max'] = np.log(df['N_est_max'])  # Max bound log

# Create figure
fig = go.Figure()

# List of unique regions to plot
regions = df['region_name'].unique()

# # Colors for each line and their respective confidence interval fill
# colors = {
#     "Northern China": ("blue", "rgba(173,216,230,0.5)"),  # Light blue
#     "Southern China": ("red", "rgba(240,128,128,0.5)")   # Light red
# }


colors = {
     "Chinese world": ("blue", "rgba(173,216,230,0.5)"),  # Light blue  # Light red
 }

# Perform LOESS smoothing and plotting
for region in regions:
    # Extract data for the region
    region_data = df[df['region_name'] == region]
    region_data.sort_values(by='decade', inplace=True)  # Sort by decade for consistent plotting

    # Applying LOESS smoothing for mean and confidence intervals
    smoothed = lowess(region_data['N_est'], region_data['decade'], frac=0.2)
    smoothed_min = lowess(region_data['N_est_min'], region_data['decade'], frac=0.2)
    smoothed_max = lowess(region_data['N_est_max'], region_data['decade'], frac=0.2)

    decades, n_est = smoothed[:, 0], smoothed[:, 1]
    n_est_min = smoothed_min[:, 1]
    n_est_max = smoothed_max[:, 1]

    # Main line
    fig.add_trace(go.Scatter(
        x=decades,
        y=n_est,
        mode='lines',
        name=f"{region}",
        line=dict(color=colors[region][0], width=3),
          showlegend=False
    ))

    ## Add original lines
      # Main line
    # fig.add_trace(go.Scatter(
    #     x=decades,
    #     y=list(region_data['N_est']),
    #     mode='lines',
    #     name=f"{region}",
    #     line=dict(color=colors[region][0], width=3),
    #       showlegend=False
    # ))

    # Confidence interval fill
    fig.add_trace(go.Scatter(
        x=np.concatenate([decades, decades[::-1]]),  # x and then x reversed for fill
        y=np.concatenate([n_est_min, n_est_max[::-1]]),  # upper bound and lower bound reversed
        fill='toself',
        fillcolor=colors[region][1],
        line=dict(color='rgba(255,255,255,0)'),  # Transparent line for fill area
        hoverinfo="skip",
        showlegend=False
    ))

# Calculate the maximum score across all individuals in the dataset
max_global_score = df_individuals['score'].max()

for region in regions:
    region_individuals = df_individuals[df_individuals['region_name'] == region]
    region_individuals = region_individuals.sort_values(by='score', ascending=False)  # Sort by score

    # Add points for all individuals
    fig.add_trace(go.Scatter(
        x=region_individuals['decade'],
        y=region_individuals['score'] / max_global_score * max(n_est),  # Normalized score
        mode='markers',
    #    marker=dict(
    #         color=colors[region][0],
    #         opacity=0.3  # Adjusted opacity for more transparency
    #     )
        marker=dict(
            color='LightCoral',
            opacity=0.3  # Adjusted opacity for more transparency
        ),
        showlegend=False
    ))

    # Add labels only for the top 10 individuals
    top_individuals = region_individuals.head(15)
    fig.add_trace(go.Scatter(
        x=top_individuals['decade'],
        y=top_individuals['score'] / max_global_score * max(n_est),
        mode='markers+text',
        name="",  # Hide name from legend
        text=top_individuals['individual_name'],
        textposition="top center",
        # textfont=dict(color=colors[region][0]),
        # marker=dict(color=colors[region][0]),
        textfont=dict(color='LightCoral'),
        marker=dict(color='LightCoral'),
        showlegend=False
    ))


# Period data
period_data = {
    "x_intercepts": [
        [-800, -476],
        [-476, -206],
        [-206, 220],
        [220, 581],
        [581, 960],
        [960, 1368],
        [1368, 1644],
        [1644, 1800]
    ],
    "labels": [
        "Spring and Autumn period",
        "Warring States period",
        "Han dynasty",
        "Six Dynasties",
        "Tang dynasty",
        "Song & Yuan dynasties",
        "Ming dynasty",
        "Qing dynasty"
    ]
}

# Calculate midpoints and add annotations
for intercept, label in zip(period_data["x_intercepts"], period_data["labels"]):
    start, end = intercept
    midpoint = (intercept[0] + intercept[1]) / 2  # Calculate the midpoint of each period
    fig.add_annotation(
        x=midpoint, y=6,  # Position the label at the top of the plot
        text=label,  # The label text
        showarrow=False,
        yshift=10,  # Shift the label slightly upwards for better visibility
        textangle=-90,  # Rotate text for vertical alignment
        font=dict(size=15, color = 'grey')
    )

    # Add vertical line at the start of each period
    fig.add_shape(
        type="line",
        x0=start, y0=0, x1=start, y1=max(df['N_est_max']),
        line=dict(
            color="lightgrey",
            width=2,
            dash="dot"  # Dotted line
        )
    )


# Update layout
fig.update_layout(
    title='',
    xaxis_title='',
    yaxis_title='Corrected CPI (Log Scale)',
    yaxis_title_font=dict(size=20),  # Increase the font size of the y-axis label,
    yaxis=dict(
        tickfont=dict(size=18)),
    template='plotly_white',  # Use a clean and clear template
    width=1200,  # Width in pixels
    height=900,   # Height in pixels
     xaxis=dict(
        tickfont=dict(size=18),
                 dtick=200,
                tickangle=45,  # Set tick angle to 45 degrees
        range=[-850, 1800]  # Set x-axis range to stop at 1800  # Set x-axis ticks every 100 units
 # Increase the font size of x-axis ticks
    ),
    
)

fig.update_layout(

  xaxis=dict(
        showgrid=False,  # Hide x-axis gridlines
        zeroline=False   # Hide the x=0 line
    ),
    yaxis=dict(
        showgrid=False,
           zeroline=False   # Hide the x=0 line  # Hide y-axis gridlines
    ),
      margin=dict(l=100, r=100, t=100, b=100)  # Set 
)


fig.show()
